In [235]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns

from tqdm import tqdm

pd.set_option('display.max_rows', 500)

In [256]:
def load_data(url): 
    df = pd.read_excel(url)
    # renames 
    df.columns = ['test_id', 'sample_id', 'li', 'be', 'nb', 'ta', 'rb', 'cs', 'w', 'bi', 'mo', 'sn']
    
    return df

df = load_data('../data/data.xls')
print(df.shape)
df.head()

(4116, 12)


,test_id,sample_id,li,be,nb,ta,rb,cs,w,bi,mo,sn
0,241810001,T01-01,90.833919,6.020064,12.937237,1.815527,162.849654,18.378212,4.631767,0.397267,1.781480,3.571594
1,241810002,T01-02,127.633179,2.942495,10.297203,1.211617,156.362156,40.746861,1.720836,0.415387,1.195794,2.528272
2,241810003,T01-03,154.726568,4.625141,10.955438,1.251676,159.181327,43.137307,7.263035,0.401448,1.091368,4.156266
3,241810004,T01-04,90.041636,2.282445,8.767717,0.885393,144.621996,29.988116,2.237632,0.423005,1.407746,3.760484
4,241810005,T01-05,67.564637,1.789445,8.592548,1.026509,92.556313,15.632865,2.381405,0.262889,1.026018,2.396971


In [261]:
def find_duplicated_rows(df, col): 
    counts_df = pd.DataFrame(df[col].value_counts())
    counts_df = counts_df.reset_index() 
    counts_df.columns = [col, f'{col}_counts']
    
    df = pd.merge(df, counts_df, on=col, how='left', validate='m:1')
    
    # remove the none duplicated rows
    dup_df = df.query(f'{col}_counts > 1')[[col, f'{col}_counts']]
    # reset the index and add by 2 to compensate the headers and starting from 0
    dup_df = dup_df.reset_index() 
    dup_df['index'] = dup_df['index'] + 2 
    
    # remove the duplicated rows with low precision
    # low precision rows can be divided by 10 without reset parts
    dup_df[f'{col}_1000'] = dup_df[col] * 1000
    dup_df[f'{col}_10'] = dup_df[f'{col}_1000'] % 10
    dup_df = dup_df.query(f'{col}_10 != 0')
    dup_df = dup_df.reset_index(drop=True)
    
    return dup_df 

dup_df = find_duplicated_rows(df, 'ta')
print(dup_df.shape)
dup_df.head()

(206, 5)


,index,ta,ta_counts,ta_1000,ta_10
0,814,2.619584,2,2619.584193,9.584193
1,816,1.049977,2,1049.977168,9.977168
2,817,2.382405,2,2382.404643,2.404643
3,818,1.091602,2,1091.602116,1.602116
4,819,0.897486,2,897.485775,7.485775


In [285]:
def find_duplicated_sequences(df, length, col):
    # Create a dictionary to store sequences and their indices
    sequences = {}
    
    # Iterate through the DataFrame to find sequences
    for start in range(len(df)):
        seq = tuple(df[col][start:start + length])
        if seq in sequences:
            sequences[seq].append(start)
        else:
            sequences[seq] = [start]
    
    # Find duplicates
    duplicates = {seq: indices for seq, indices in sequences.items() if len(indices) > 1}
    
    return duplicates

In [286]:
dup_df = find_duplicated_rows(df, 'li')
print(dup_df.shape)

(0, 5)


In [299]:
cols = df.columns[2:]
for col in cols: 
    print(f'----------{col}----------')
    dup_df = find_duplicated_rows(df, col)
    duplicated_sequences = []
    length = 0
    if dup_df.shape[0] > 0: 
        # Find and print duplicated sequences
        for length_tmp in tqdm(range(2, len(dup_df))): 
            duplicated_sequences_temp = find_duplicated_sequences(dup_df, length_tmp, col)
            if len(duplicated_sequences_temp) == 1: 
                duplicated_sequences = duplicated_sequences_temp
                length = length_tmp
                for seq, indices in duplicated_sequences.items():
                    print(f'{length:}')
                    print(f"Indices: {indices}, Sequence: {seq}")
                    idx1, idx2, *_ = indices 
                    starting_rows1 = dup_df.iloc[idx1, :]['index']
                    starting_rows2 = dup_df.iloc[idx2, :]['index']
                    # ending_rows1 = starting_rows1 + length - 1
                    # ending_rows2 = starting_rows2 + length - 1
                    ending_rows1 = dup_df.iloc[idx1+length-1, :]['index']
                    ending_rows2 = dup_df.iloc[idx2+length-1, :]['index']
                    print(f'rows from {starting_rows1} to {ending_rows1}')
                    print(f'rows from {starting_rows2} to {ending_rows2}')
            elif len(duplicated_sequences_temp) > 1: 
                print(f'{len(duplicated_sequences_temp)}: {length_tmp}')         
    else: 
        print(f'There is no duplicated rows in column {col}.')

----------li----------
There is no duplicated rows in column li.
----------be----------


100%|██████████| 11/11 [00:00<00:00, 4038.28it/s]

2: 2
----------nb----------


100%|██████████| 6/6 [00:00<00:00, 4640.57it/s]


2
Indices: [0, 6], Sequence: (13.600000000000001, 16.1)
rows from 3851.0 to 3920.0
rows from 4061.0 to 4106.0
----------ta----------


...

100%|██████████| 3/3 [00:00<00:00, 5405.03it/s]


----------mo----------


100%|██████████| 6/6 [00:00<00:00, 5886.74it/s]


----------sn----------


100%|██████████| 5/5 [00:00<00:00, 5301.19it/s]
